In [1]:
#!fsharp
#!value --from-file input.txt --name input

In [1]:
#!fsharp
#!share input --from value

type Instruction = 
  | Jump of int
  | Acc of int
  | NoOp of int

let parseProgram (inp:string) = 
  inp.Trim().Split('\n')
  |> Array.map (fun instruction -> 
    match instruction.Split(' ') with
    | [| "jmp"; i |] -> Jump (int i)
    | [| "acc"; i |] -> Acc (int i)
    | [| "nop"; i |] -> NoOp (int i)
    | err -> failwith $"Bad instruction: '%A{err}'"
    )

let exampleProgram = parseProgram """nop +0
acc +1
jmp +4
acc +3
jmp -3
acc -99
acc +1
jmp -4
acc +6"""
let fullProgram = parseProgram input

exampleProgram |> sprintf "%A"

[|NoOp 0; Acc 1; Jump 4; Acc 3; Jump -3; Acc -99; Acc 1; Jump -4; Acc 6|]

In [1]:
#!fsharp
type State = 
  {
    Head: int
    Acc: int
    Visited: int Set
  }

type MachineState =
  | Running of State
  | Looping of State
  | Error of int
  | Halted of int

let initialState = Running { Head = 0; Acc = 0; Visited = Set.empty }
let runProgram (program: Instruction array) = 
  let setNextHead state i = { state with Head = state.Head + i; Visited = state.Visited.Add state.Head }
  let runInstruction = function
    | Running { Head = head; Acc = acc } when head = program.Length -> Halted acc |> Some
    | Running { Head = head } when head > program.Length || head < 0 -> Error head |> Some
    | Running ({ Head = head; Visited = visited } as state) when visited |> Set.contains head -> Looping state |> Some
    | Running ({ Head = head; Acc = acc; Visited = visited } as state) ->
      let next = 
        match program.[head] with 
        | NoOp _ -> setNextHead state 1
        | Acc i -> setNextHead { state with Acc = acc + i } 1
        | Jump i -> setNextHead state i
      next |> Running |> Some
    | _ -> None
  initialState |> Seq.unfold (runInstruction >> Option.map (fun x -> (x,x)))

runProgram exampleProgram 
|> Seq.last 
|> printf "Example program finishes execution with state %A"

runProgram fullProgram 
|> Seq.last
|> printf "Full program finishes execution with state %A"

Example program finishes execution with state 

Looping { Head = 1
          Acc = 5
          Visited = set [0; 1; 2; 3; 4; 6; 7] }

Full program finishes execution with state 

Looping { Head = 336
          Acc = 1610
          Visited = set [0; 1; 2; 3; 4; 28; 29; 30; 31; ...] }

In [1]:
#!fsharp
let flipInstruction = function
  | Jump i -> NoOp i
  | NoOp i -> Jump i
  | x -> x

let flipInstructionAt i = Array.mapi (fun j -> if j = i then flipInstruction else id)

Seq.init fullProgram.Length id
|> Seq.map (fun i -> 
  fullProgram
  |> flipInstructionAt i
  |> runProgram
  |> Seq.last
)
|> Seq.indexed
|> Seq.pick (function 
  | i, (Halted _ as haltState) -> Some $"Flipping instruction '%i{i}: %A{fullProgram.[i]}' allows program to successfully finish with state %A{haltState}" 
  | _ -> None)

Flipping instruction '188: Jump -55' allows program to successfully finish with state Halted 1703